# Дисклеймер 
Целью данной работы было исключительно изучение использования пайплайна и использования самописных классов для преобразования данных в пайплайне. Так же в работе были построены эмпириеские доверительные интервалы для сравнения результатов различных моделей.
<br>
В данной работе я не приследовал цель достичь максимально точных прогнозов, поэтому не проводил серьзного EDA, не расматривал precession, recall и f-metric, не балансировал датасет и др.

## Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import roc_auc_score

import seaborn as sns

In [2]:
df = pd.read_csv('Churn_Modelling.csv')

## Первичное изучение данных

##### Смотрим типы данных по колонкам, размер датасета и количество пропущенных значений

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


##### Разделяем колонки по типам данных

In [4]:
cat = df.select_dtypes(include=['object']).columns

In [5]:
num = df.select_dtypes(include=['int64','float64']).columns

In [6]:
cat

Index(['Surname', 'Geography', 'Gender'], dtype='object')

In [7]:
cat = cat[1:]
cat

Index(['Geography', 'Gender'], dtype='object')

In [8]:
num

Index(['RowNumber', 'CustomerId', 'CreditScore', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
       'Exited'],
      dtype='object')

In [9]:
num = num[2:-1]
num

Index(['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary'],
      dtype='object')

### Категориальные переменнные

In [10]:
df[cat].nunique()

Geography    3
Gender       2
dtype: int64

### Количественные переменные

##### Смотрим базовые статистики

In [11]:
df.iloc[:,2:].describe()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


Данные кажутся довольно простыми, из явных осложняющих факторов - разбалансировка датасета (более 75% имеют целевую переменную 0),  построим первичную модель по данным as-is

стоит так же отметить:<br>
не менее 25% имеют нулевой баланс, возможно это только кредитные клиенты, возможно это отток или что то еще <br>

В целом данные кажутся пригодными для построения модели AS-IS

##  Построение базовой модели на данных AS-IS

##### Создадим классы для удобства построения пайплайна

In [12]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    """
    Трансформер для выбора нечисловых колонок
    """
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Трансформер для выбора числовых колонок
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEncoder(BaseEstimator, TransformerMixin):
    """Трансформер для ONE HOT ENCODING"""
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns[:-1]]

### Создание пайплайна

##### Создадим список  трансформеров, преобразующих данные

In [13]:
final_transformers = list()
#Категориальные переменные выбираются и подвергаются OHE
for cat_col in cat:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEncoder(key=cat_col))
            ])
    final_transformers.append((cat_col, cat_transformer))
#Количественные переменные выбираются и подвергаются масштабированию
for cont_col in num:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                 ('scaler', StandardScaler())
            ])
    final_transformers.append((cont_col, cont_transformer))

In [14]:
final_transformers

[('Geography',
  Pipeline(steps=[('selector', FeatureSelector(column='Geography')),
                  ('ohe', OHEncoder(key='Geography'))])),
 ('Gender',
  Pipeline(steps=[('selector', FeatureSelector(column='Gender')),
                  ('ohe', OHEncoder(key='Gender'))])),
 ('CreditScore',
  Pipeline(steps=[('selector', NumberSelector(key='CreditScore')),
                  ('scaler', StandardScaler())])),
 ('Age',
  Pipeline(steps=[('selector', NumberSelector(key='Age')),
                  ('scaler', StandardScaler())])),
 ('Tenure',
  Pipeline(steps=[('selector', NumberSelector(key='Tenure')),
                  ('scaler', StandardScaler())])),
 ('Balance',
  Pipeline(steps=[('selector', NumberSelector(key='Balance')),
                  ('scaler', StandardScaler())])),
 ('NumOfProducts',
  Pipeline(steps=[('selector', NumberSelector(key='NumOfProducts')),
                  ('scaler', StandardScaler())])),
 ('HasCrCard',
  Pipeline(steps=[('selector', NumberSelector(key='HasCrCard')),


##### Объединим список трансформеров в один трансформер

In [15]:
feats = FeatureUnion(final_transformers)

feature_processing = Pipeline([('feats', feats)])

#####  Добавим классификатор (любой, позже классификатор будет меняться как настраиваемый параметр)

In [16]:
pipeline = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression()),
])

### Логистическая регрессия

##### Создадим сетку параметров логистической регрессии для gridsearch cv

In [17]:
param_grid = [
    {'classifier':[LogisticRegression()],
     'classifier__penalty':[ 'l2'],
     'classifier__solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
     },
    
    {'classifier':[LogisticRegression()],
     'classifier__penalty':['l1'],
     'classifier__solver':[ 'liblinear', 'saga']
     },
    
    {'classifier':[LogisticRegression()],
     'classifier__penalty':['none'],
     'classifier__solver':[ 'newton-cg', 'lbfgs', 'sag', 'saga']
     },
    
    {'classifier':[LogisticRegression()],
     'classifier__penalty':['elasticnet'],
     'classifier__solver':['saga'],
     'classifier__l1_ratio':[0.01,0.05,0.1,0.3,0.5,0.7, 0.9]
     }
    
    
    
]

##### Подберем лучшие параметры для пайплайна с классификатором-логистической регрессией

In [18]:
grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=16, 
                           scoring='roc_auc',
                          #refit = 'roc_auc', 
                           n_jobs = -1)

In [19]:
grid_search.fit(df,df['Exited'])

GridSearchCV(cv=16,
             estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('Geography',
                                                                        Pipeline(steps=[('selector',
                                                                                         FeatureSelector(column='Geography')),
                                                                                        ('ohe',
                                                                                         OHEncoder(key='Geography'))])),
                                                                       ('Gender',
                                                                        Pipeline(steps=[('selector',
                                                                                         FeatureSelector(column='Gender')),
                                                                                        ('o

In [20]:
results  = tuple(zip(grid_search.cv_results_['params'],grid_search.cv_results_['mean_test_score'],grid_search.cv_results_['std_test_score']))

In [21]:
for result in sorted(results, key = lambda x: x[1], reverse = True):
    print('')
    print(result)


({'classifier': LogisticRegression(solver='liblinear'), 'classifier__penalty': 'l2', 'classifier__solver': 'liblinear'}, 0.7658373940335778, 0.016766120160210965)

({'classifier': LogisticRegression(), 'classifier__penalty': 'l1', 'classifier__solver': 'liblinear'}, 0.7658344811471126, 0.01673985266156785)

({'classifier': LogisticRegression(), 'classifier__l1_ratio': 0.3, 'classifier__penalty': 'elasticnet', 'classifier__solver': 'saga'}, 0.7658315165327496, 0.016757347853326738)

({'classifier': LogisticRegression(), 'classifier__l1_ratio': 0.1, 'classifier__penalty': 'elasticnet', 'classifier__solver': 'saga'}, 0.7658266099940765, 0.016750881669039357)

({'classifier': LogisticRegression(), 'classifier__l1_ratio': 0.5, 'classifier__penalty': 'elasticnet', 'classifier__solver': 'saga'}, 0.7658217092029476, 0.01674865443502603)

({'classifier': LogisticRegression(), 'classifier__l1_ratio': 0.01, 'classifier__penalty': 'elasticnet', 'classifier__solver': 'saga'}, 0.7658196983081071, 0

In [22]:
grid_search.best_params_

{'classifier': LogisticRegression(solver='liblinear'),
 'classifier__penalty': 'l2',
 'classifier__solver': 'liblinear'}

In [23]:
grid_search.best_score_

0.7658373940335778

### Градиентный бустинг

##### Создаем сетку параметров  для градиентного бустинг

In [24]:
param_grid = [
    {'classifier':[GradientBoostingClassifier()],
     'classifier__n_estimators':[ 50, 100, 200, 500],
     'classifier__max_depth':[1,5,10,15]
     
     }  
]

In [25]:
grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=16, 
                           scoring='roc_auc',
                          #refit = 'roc_auc', 
                           n_jobs = -1)

In [26]:
grid_search.fit(df,df['Exited'])

GridSearchCV(cv=16,
             estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('Geography',
                                                                        Pipeline(steps=[('selector',
                                                                                         FeatureSelector(column='Geography')),
                                                                                        ('ohe',
                                                                                         OHEncoder(key='Geography'))])),
                                                                       ('Gender',
                                                                        Pipeline(steps=[('selector',
                                                                                         FeatureSelector(column='Gender')),
                                                                                        ('o

In [27]:
results  = tuple(zip(grid_search.cv_results_['params'],grid_search.cv_results_['mean_test_score'],grid_search.cv_results_['std_test_score']))

In [28]:
for result in sorted(results, key = lambda x: x[1], reverse = True):
    print('')
    print(result)


({'classifier': GradientBoostingClassifier(max_depth=5, n_estimators=50), 'classifier__max_depth': 5, 'classifier__n_estimators': 50}, 0.8654132811226717, 0.013321220898949566)

({'classifier': GradientBoostingClassifier(max_depth=5, n_estimators=50), 'classifier__max_depth': 5, 'classifier__n_estimators': 100}, 0.8651299556675625, 0.015500388915099873)

({'classifier': GradientBoostingClassifier(max_depth=5, n_estimators=50), 'classifier__max_depth': 5, 'classifier__n_estimators': 200}, 0.8590636001060556, 0.015564460452342615)

({'classifier': GradientBoostingClassifier(max_depth=5, n_estimators=50), 'classifier__max_depth': 1, 'classifier__n_estimators': 500}, 0.8519743802945408, 0.013712156179586607)

({'classifier': GradientBoostingClassifier(max_depth=5, n_estimators=50), 'classifier__max_depth': 1, 'classifier__n_estimators': 200}, 0.8507951163107541, 0.013463046415989203)

({'classifier': GradientBoostingClassifier(max_depth=5, n_estimators=50), 'classifier__max_depth': 5, 'cl

In [29]:
grid_search.best_params_

{'classifier': GradientBoostingClassifier(max_depth=5, n_estimators=50),
 'classifier__max_depth': 5,
 'classifier__n_estimators': 50}

In [30]:
grid_search.best_score_

0.8654132811226717

##### Лучшие параметры, лучший скор, стандартное отклонение скора

In [37]:
sorted(results, key = lambda x: x[1], reverse = True)[0]

({'classifier': GradientBoostingClassifier(max_depth=5, n_estimators=50),
  'classifier__max_depth': 5,
  'classifier__n_estimators': 50},
 0.8654031749205715,
 0.01323004871746928)

базовые модели показывают довольно неплохие результаты

##### Проверка лучшей базовой модели, фиксация ее метрик качества

In [38]:
clf = pipeline.set_params(**
                          {'classifier': GradientBoostingClassifier(max_depth=5, n_estimators=50, random_state = 0),
                           'classifier__max_depth': 5,
                           'classifier__n_estimators': 50}
                         )

In [39]:
X_train, X_test, y_train, y_test = train_test_split(df, df['Exited'], random_state=0)

In [40]:
clf.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Geography',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEncoder(key='Geography'))])),
                                                ('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEncoder(key='Gender'))])),
                                                ('CreditScore',
                                                 Pipeline(steps=[('selector',
        

In [41]:
clf.score(X_test, y_test)

0.87

In [42]:
roc_auc_score(y_test, clf.decision_function(X_test))

0.8747502267078079

проверка подтвердила результаты кросс валидации

### Более точный подбор параметров модели (Построение улучшенной модели)

In [260]:
param_grid = [
    {'classifier':[GradientBoostingClassifier()],
     'classifier__n_estimators':[ 30, 50, 60],
     'classifier__max_depth':[3,4,5]
     
     }  
]

In [282]:
param_grid = [
    {'classifier':[GradientBoostingClassifier()],
     'classifier__n_estimators':[50, 1000, 2000],
     'classifier__max_depth':[3,4,5,6,7],
     'classifier__learning_rate':[0.1,0.05,0.01]
     
     }  
]

In [283]:
grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=16, 
                           scoring='roc_auc',
                          #refit = 'roc_auc', 
                           n_jobs = -1)

In [284]:
grid_search.fit(df,df['Exited'])

GridSearchCV(cv=16,
             estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('Geography',
                                                                        Pipeline(steps=[('selector',
                                                                                         FeatureSelector(column='Geography')),
                                                                                        ('ohe',
                                                                                         OHEncoder(key='Geography'))])),
                                                                       ('Gender',
                                                                        Pipeline(steps=[('selector',
                                                                                         FeatureSelector(column='Gender')),
                                                                                        ('o

In [285]:
results  = tuple(zip(grid_search.cv_results_['params'],grid_search.cv_results_['mean_test_score'],grid_search.cv_results_['std_test_score']))

In [286]:
grid_search.best_params_

{'classifier': GradientBoostingClassifier(learning_rate=0.01, n_estimators=2000),
 'classifier__learning_rate': 0.01,
 'classifier__max_depth': 3,
 'classifier__n_estimators': 2000}

In [287]:
grid_search.best_score_

0.8672860311598178

In [288]:
for result in sorted(results, key = lambda x: x[1], reverse = True):
    print('')
    print(result)


({'classifier': GradientBoostingClassifier(learning_rate=0.01, n_estimators=2000), 'classifier__learning_rate': 0.01, 'classifier__max_depth': 3, 'classifier__n_estimators': 2000}, 0.8672860311598178, 0.014294696346008022)

({'classifier': GradientBoostingClassifier(learning_rate=0.01, n_estimators=2000), 'classifier__learning_rate': 0.01, 'classifier__max_depth': 3, 'classifier__n_estimators': 1000}, 0.8666572182911936, 0.013942771077807678)

({'classifier': GradientBoostingClassifier(learning_rate=0.01, n_estimators=2000), 'classifier__learning_rate': 0.01, 'classifier__max_depth': 4, 'classifier__n_estimators': 1000}, 0.8664458083769869, 0.014064296182898495)

({'classifier': GradientBoostingClassifier(learning_rate=0.01, n_estimators=2000), 'classifier__learning_rate': 0.1, 'classifier__max_depth': 5, 'classifier__n_estimators': 50}, 0.8663459740158918, 0.013539394567718138)

({'classifier': GradientBoostingClassifier(learning_rate=0.01, n_estimators=2000), 'classifier__learning_r

In [289]:
clf = pipeline.set_params(**
                          {'classifier': GradientBoostingClassifier(learning_rate=0.01, n_estimators=2000),
                           'classifier__learning_rate': 0.01,
                           'classifier__max_depth': 3,
                           'classifier__n_estimators': 2000}
                         )

In [290]:
X_train, X_test, y_train, y_test = train_test_split(df, df['Exited'], random_state=0)

In [291]:
clf.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Geography',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEncoder(key='Geography'))])),
                                                ('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEncoder(key='Gender'))])),
                                                ('Tenure',
                                                 Pipeline(steps=[('selector',
             

In [292]:
clf.score(X_test, y_test)

0.8668

In [293]:
roc_auc_score(y_test, clf.decision_function(X_test))

0.8751982151508902

Получиось добиться некоторого улучшения качества, однако, оно скорее всего незначимо

In [296]:
roc_auc_score(y_test, clf.decision_function(X_test))

0.8751982151508902

### Построение эмпирического доверитльного интервала

##### Генерирум разные ROC_AUC для данной модели на бутсрапах

In [311]:
import warnings
warnings.filterwarnings("ignore")

from scipy.stats import sem



y_pred = clf.decision_function(X_test)
y_true = np.array(y_test)


print("Исходный ROC_AUC: {:0.3f}".format(roc_auc_score(y_true, y_pred)))


n_bootstraps = 1000
rnd_seed = 0  
bootstrapped_scores = []


rnd = np.random.RandomState(rnd_seed)
for i in range(n_bootstraps):
    # bootstrap by sampling with replacement on the prediction indices
    indices = rnd.random_integers(0, len(y_pred) - 1, len(y_pred))
    if len(np.unique(y_true[indices])) < 2:
        # We need at least one positive and one negative sample for ROC AUC
        # to be defined: reject the sample
        continue


    score = roc_auc_score(y_true[indices], y_pred[indices])
    bootstrapped_scores.append(score)
    #print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))

Исходный ROC_AUC: 0.875


In [313]:
np.min(bootstrapped_scores), np.max(bootstrapped_scores), len(bootstrapped_scores)

(0.8460889263980811, 0.9015676270391936, 1000)

#####  Строим 95% доверительный интервал по бутстрапированным roc auc

In [323]:
conf_int = np.percentile(bootstrapped_scores, (2.5, 97.5))
conf_int

array([0.85713737, 0.89129335])

In [324]:
#средняя оценка качества лучшей из базовых моделей на градиентном бустинге
basic_bosting_roc_auc = 0.8654031749205715

In [328]:
min(conf_int)< basic_bosting_roc_auc and basic_bosting_roc_auc<max(conf_int)

True

улучшенная бустинговая модель хоть не имеет статистически значимого улучшения относительно базовой

In [325]:
#cредняя оценка качества лучшей из базовых логистических регрессий
basic_logreg_roc_auc = 0.7658373940335778 

In [331]:
min(conf_int)<= basic_logreg_roc_auc 

False

улучшенная бустинговая модель значимо (на 95% доверительном интервале) лучше чем базовая модель на логистической регрессии